In [1]:

import sys
import os
sys.path.append(os.path.abspath("../..")) 

from deap import base, creator, tools, benchmarks
import deap.cma as cma

import copy
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import math
import time
import pandas as pd

from Modules.Helpers import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotter
from Modules.Equations import Equation
from Modules.Models import *

In [2]:
np.random.seed(42)
GRN5 = ModelWrapper.GRN5()

In [3]:
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='../../Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])
t_eval = np.array(df['t'])
original = np.array(df[labels]).T
bounds = {
    'tau': (0.1, 5.0),
    'k': (0.1, 2.0),
    'n': (0.1, 30.0),
    'strategy': (0.1, 10)
}

IND_SIZE = 19


In [4]:
# labels = GRN5.labels
# df, max_data = GRN5.df, GRN5.max_data
# initial_conditions = GRN5.initial_conditions
# t_span = GRN5.t_span
# t_eval = GRN5.t_eval
# original = GRN5.original
# bounds = GRN5.bounds
# IND_SIZE = GRN5.IND_SIZE

In [5]:
class Coefficient:
    def __init__(self, bounds_val, bounds_strg):
        self.val = random.uniform(*bounds_val)
        self.bounds_val = bounds_val
    
    def __repr__(self):
        return f"val={self.val}"

class CMACoefficient:
    def __init__(self, val, bounds_val):
        self.bounds_val = bounds_val
        self.val = self.limit_val(val)
    
    def limit_val(self, val):
        return max(self.bounds_val[0], min(val, self.bounds_val[1]))
    
    def __repr__(self):
        return f"val={self.val}"

class Individual:
    def __init__(self):
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': True},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None, '-': False},
                'B': {'n': None, 'k': None, '-': False},
                'E': {'n': None, 'k': None, '-': False},
                'tau': None,
            }
        }
        self.ind_size = IND_SIZE
        self.fitness = np.inf
        # self.fitness = random.uniform(0, 1)
    
    @staticmethod
    def list_to_ind(list_ind):
        i = 0
        ind = Individual()
        for key, label in ind.coeffs.items():
            label['tau'] = CMACoefficient(list_ind[i], bounds['tau'])
            i += 1
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = CMACoefficient(list_ind[i], bounds['n'])
                    coeffs['k'] = CMACoefficient(list_ind[i+1], bounds['k'])
                    i += 2
        return ind
    
    def ind_to_list(self):
        ind_list = []
        for key, label in self.coeffs.items():
            ind_list.append(label['tau'].val)
            for key, coeffs in label.items():
                if key != 'tau':
                    ind_list.append(coeffs['n'].val)
                    ind_list.append(coeffs['k'].val)
        return ind_list
    
    @staticmethod
    def apply_bounds(population):
        for ind in population:
            list_ind = Individual.list_to_ind(ind)
            ind[:] = Individual.ind_to_list(list_ind)
    
    @staticmethod    
    def cma_evaluate(list_ind):
        ind = Individual.list_to_ind(list_ind)
        ind.calc_fitness()
        return ind.fitness,
        # return ind
    
    def calc_fitness(self, method='RK45'):
        try:
            equation = Equation(self.numerical_coeffs, labels)
            y = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, )).y
            self.fitness = self.squared_error(original, y)
            self.fitness = min(self.fitness, 1e6)
        except:
            print("Overflow")
            self.fitness = 1e6
       
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]
    
    @staticmethod
    def abs_error(original, pred):
        return sum(sum(abs(original-pred)))
    
    @staticmethod
    def squared_error(original, pred):
        return sum(sum( (original-pred)**2 ))**(1/2)
    
    @staticmethod
    def MSE_error(original, pred):
        return np.mean((original-pred)**2)
    
    @staticmethod
    def mean_abs_error(original, pred):
        return np.mean(abs(original-pred))
            
    @staticmethod
    def initialize_ind(bounds):
        ind = Individual()
        for key, label in ind.coeffs.items():
            label['tau'] = Coefficient(bounds['tau'], bounds['strategy'])
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = Coefficient(bounds['n'], bounds['strategy'])
                    coeffs['k'] = Coefficient(bounds['k'], bounds['strategy'])
                    
        ind.calc_fitness()
        return ind
    
    @property
    def numerical_coeffs(self,):
        
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for key, label in numerical_coeffs.items():
            label['tau'] = label['tau'].val
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = int(coeffs['n'].val)
                    coeffs['k'] = coeffs['k'].val
                    
        return numerical_coeffs
    
    
    def plot(self, method='RK45', comparison=True):
        methods = [method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, )).y
        Plotter.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        if comparison:
            Plotter.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
            
    @staticmethod        
    def initialize_average_bounds(bounds, ind_size):
        averages = [np.mean(value) for value in bounds.values()]
        array = np.resize(averages, ind_size)
        return array
    
    @staticmethod
    def init_test(model):
        array = np.zeros(model.IND_SIZE)
        i = 0
        for key, label in model.coeffs.items():
            array[i] = np.mean(model.bounds['tau'])
            i += 1
            for key, coeffs in label.items():
                if key != 'tau':
                    array[i] = np.mean(model.bounds['n'])
                    array[i+1] = np.mean(model.bounds['k'])
                    i += 2
                    
        return array
        
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual(fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"

In [6]:
# Criação do tipo de indivíduo e toolbox
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("evaluate", Individual.cma_evaluate)

# Inicializa estratégia CMA-ES
centroids = Individual.init_test(GRN5)
strategy = cma.Strategy(centroid=centroids, sigma=10, lambda_=int(4+(3*np.log(IND_SIZE))))

toolbox.register("generate", strategy.generate, creator.Individual)
toolbox.register("update", strategy.update)

# Estatísticas e parâmetros do algoritmo
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda individual: individual.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [7]:
centroids

array([ 2.55, 15.05,  1.05,  2.55, 15.05,  1.05,  2.55, 15.05,  1.05,
        2.55, 15.05,  1.05,  2.55, 15.05,  1.05, 15.05,  1.05, 15.05,
        1.05])

In [ ]:
GENERATION_LIMIT = 5000
NO_IMPROVEMENT_LIMIT = 50
HARD_SIGMA_INCREASE_FACTOR = 10
TOLERANCE = 1E-4


population = toolbox.generate()
Individual.apply_bounds(population)

best_fitness = None
no_improvement_counter = 0
best_individual = None

for gen in range(GENERATION_LIMIT):
    for i, ind in enumerate(population):
        ind.fitness.values = toolbox.evaluate(ind)

    record = stats.compile(population)

    current_best_fitness = min(ind.fitness.values[0] for ind in population)
    hof.update(population)

    if best_fitness is None or current_best_fitness < best_fitness - TOLERANCE:
        best_fitness = current_best_fitness
        no_improvement_counter = 0
        best_individual = hof[0]
    else:
        no_improvement_counter += 1

    if no_improvement_counter >= (NO_IMPROVEMENT_LIMIT + gen/100):
        
        strategy.sigma = min(max(strategy.sigma*HARD_SIGMA_INCREASE_FACTOR, 1), 15)
        print(f"Sigma increased due to no improvement. New sigma: {strategy.sigma}")
        no_improvement_counter = 0

    toolbox.update(population)

    print(f"Generation {gen}: {record}", end='\r')

    if gen % 500 == 0:
        best_ind = Individual.list_to_ind(best_individual)

    population = toolbox.generate()
    Individual.apply_bounds(population)

    # population[0][:] = super_ind  # e.g., injecting the best individual into the population


Sigma increased due to no improvement. New sigma: 1.5123260448571516, 'min': 7.4938906855164005, 'max': 13.078528065569849}}}
Sigma increased due to no improvement. New sigma: 2.3657608246267982, 'min': 7.861940106329958, 'max': 9.26259412389079}8}}
Sigma increased due to no improvement. New sigma: 5.3090502932475294, 'min': 8.034470009380135, 'max': 13.933325964235697}}
Sigma increased due to no improvement. New sigma: 5.195462325525144, 'min': 7.8659974544450275, 'max': 12.682392586406703}}}
Sigma increased due to no improvement. New sigma: 3.12303879672377216, 'min': 7.644244288171398, 'max': 8.205509800281202}}}
Sigma increased due to no improvement. New sigma: 6.0986298188649342, 'min': 7.537371264830798, 'max': 10.07347940522684}}}
Sigma increased due to no improvement. New sigma: 1 0.0012998999285775237, 'min': 2.629676576595791, 'max': 2.63440655143961}}}}}
Sigma increased due to no improvement. New sigma: 10.0009583752110430033, 'min': 2.434026436687267, 'max': 2.4369323424956